In [1]:
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import spacy
from tqdm import tqdm

In [2]:
#execute this if the model and sapacy aren't installed
%pip install spacy
%python -m spacy download pt_core_news_sm

Note: you may need to restart the kernel to use updated packages.


UsageError: Line magic function `%python` not found (But cell magic `%%python` exists, did you mean that instead?).


In [4]:
FILE="train.txt" #change this if necessary
f=open(FILE,"r")
TRAIN_DATA=eval(f.read())

In [7]:
model = 'pt_core_news_sm'
n_iter=100

In [8]:
if model is not None:
    nlp = spacy.load(model)  
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  
    print("Created blank 'en' model")

#set up the pipeline

if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)
else:
    ner = nlp.get_pipe('ner')

Loaded model 'pt_core_news_sm'


In [16]:

examples=[]
for data in TRAIN_DATA:
    text=data['product']
    annotations_list=[]
    for i in data["tags"]:
        annotations_list.append((i[1],i[2],i[0]))
    annotations={"entities":annotations_list}
    examples.append(spacy.training.Example.from_dict(nlp.make_doc(text), annotations))
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.create_optimizer()
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        nlp.update(
            examples,
            drop=0.5,                  sgd=optimizer,
            losses=losses)
        print(losses)

{'ner': 3041.919858902693}
{'ner': 2978.503441828758}
{'ner': 2829.509812951088}
{'ner': 2771.8409024477005}
{'ner': 2756.1859478382394}
{'ner': 2620.9111109972}
{'ner': 2478.7261045724154}
{'ner': 2437.766159892082}
{'ner': 2359.4690890312195}
{'ner': 2236.3878116607666}
{'ner': 2191.9950379133224}
{'ner': 2060.944660782814}
{'ner': 1971.509877562523}
{'ner': 1949.0719864368439}
{'ner': 1944.7201251983643}
{'ner': 1970.5462651252747}
{'ner': 2021.8714714050293}
{'ner': 2048.9361933469772}
{'ner': 2030.6688047647476}
{'ner': 1983.1865845918655}
{'ner': 1919.0386760234833}
{'ner': 1826.547473192215}
{'ner': 1796.8161436319351}
{'ner': 1774.4664803743362}
{'ner': 1780.585930109024}
{'ner': 1735.6864491701126}
{'ner': 1724.712613105774}
{'ner': 1725.9679911136627}
{'ner': 1723.7937417030334}
{'ner': 1708.4213209152222}
{'ner': 1698.928923368454}
{'ner': 1672.9047198295593}
{'ner': 1642.329697549343}
{'ner': 1630.4521592855453}
{'ner': 1663.6833994984627}
{'ner': 1677.2875323295593}
{'ner'

In [18]:
#maybe add some logic to make evaluations and save the model
doc=nlp("Chá Matte Leão Original Zero 1,5l")
spacy.displacy.render(doc,style="ent",jupyter=True)